## TensorFlow TFRecord输入数据格式与图像预处理

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 7.1 TFRecord输入数据格式

一种分类方式是使用一个从类别名称到所有数据列表的词典来维护图像和类别的关系，这种方式的可扩展性非常差，很难有效地记录输入数据中的信息了。于是TensorFlow提供了一种统一的格式来存储数据——**TFRecord**。  
TFRecord文件中的数据都是通过tf.train.Example Protocol Buffer的格式存储的。

In [ ]:
# tf.train.Example 的定义
message Example {
    Feature features = 1;
};

message Features {
    map<string, Feature> feature = 1;
};

message Feature {
    oneof kind {
        BytesList bytes_list = 1;
        FloatList float_list = 2;
        Int64List int64_list = 3;
  }
};

tf.train.Example中包含了一个**从属性名称到取值的字典**。其中属性名称为一个字符串，属性的取值可以为字符串（BytesList）、实数列表（FloatList）或者整数列表（Int64List）。  
以下程序给出如何将MNIST输入数据转化为TFRecord的格式：

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 将数据转化为tf.train.Example格式。
def _make_example(pixels, label, image):
    image_raw = image.tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(label)),
        'image_raw': _bytes_feature(image_raw)
    }))
    return example

# 读取mnist训练数据。
mnist = input_data.read_data_sets("../../datasets/MNIST_data", 
                                  dtype=tf.uint8, 
                                  one_hot=True)
images = mnist.train.images
# 训练数据所对应的正确答案，作为一个属性保存在TFRecord中
labels = mnist.train.labels
# 训练数据的图像分辨率，作为Example中的一个属性
pixels = images.shape[1]
num_examples = mnist.train.num_examples

# 输出包含训练数据的TFRecord文件。
# 先创建一个writer来写TFRecord文件
with tf.python_io.TFRecordWriter(path="output.tfrecords") as writer:
    for index in range(num_examples):
        # 将一个样例转化为Example Protocol Buffer，并将所有的信息写入这个数据结构
        example = _make_example(pixels, labels[index], images[index])
        # 将一个Example写入TFRecord文件
        writer.write(example.SerializeToString())
print("TFRecord训练文件已保存。")

# 读取mnist测试数据。
images_test = mnist.test.images
labels_test = mnist.test.labels
pixels_test = images_test.shape[1]
num_examples_test = mnist.test.num_examples

# 输出包含测试数据的TFRecord文件。
with tf.python_io.TFRecordWriter(path="output_test.tfrecords") as writer:
    for index in range(num_examples_test):
        example = _make_example(
            pixels_test, labels_test[index], images_test[index])
        writer.write(example.SerializeToString())
print("TFRecord测试文件已保存。")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
TFRecord训练文件已保存。
TFRecord测试文件已保存。


以下程序给出如何读取TFRecord文件中的数据：

In [2]:
import tensorflow as tf

# 创建一个reader来读取TFRecord文件中的样例
reader = tf.TFRecordReader()
# 创建一个列表来维护输入文件列表，在7.3.2节中会详细介绍
filename_queue = tf.train.string_input_producer(["output.tfrecords"])

# 从文件中读取一个样例。也可以使用read_up_to函数一次性读取多个样例
_, serialized_example = reader.read(filename_queue)

# 解析读取的样例。如果需要解析多个样例，可使用parse_example函数
features = tf.parse_single_example(
    serialized_example,
    features={
        # TensorFlow提供两种不同的属性解析方法。
        # 一种是方法是tf.FixedLenFeature,这种方法解析的结果为一个Tensor。
        # 另一种方法是tf.VarLenFeature，这种方法得到的解析结果为SparseTensor，用于处理稀疏数据。
        # 这里解析数据的格式需要和上面程序写入数据的格式一致。
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

# tf.decode_raw函数可以将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

sess = tf.Session()
# 启动多线程处理输入数据
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# 每次运行可以读取TFRecord文件中的一个样例。当所有样例都读完之后，在此样例中程序会再重头读取
for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])
print(label, pixel, image)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
8 784 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  22  69
 148 210 253 156 122   7   0   0  18   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 100 221 252 252 253 252 252 252 113   0   0
 185   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  31 221
 252 252 244 23

### 7.2 图像数据处理

通过对图像的预处理，可以尽量避免模型受到无关因素的影响，从而提高模型的准确率。

#### 图像编码处理

图像在存储时并不是直接记录这些矩阵中的数字，而是记录经过**压缩编码**之后的结果。  
TensorFlow提供了对jpeg和png格式图像的编码/解码函数： tf.image.decode_jpeg、tf.image.decode_png、tf.image.encode_jpeg、tf.image.encode_png 。  
以下代码示范了如何使用TensorFlow中对jpeg格式图像进行编码/解码：

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf   
import numpy as np
%matplotlib inline

# 读取图像的原始数据
image_raw_data = tf.gfile.FastGFile(name="../../datasets/cat.jpg", mode='rb').read()

with tf.Session() as sess:
    # 对图像进行jpeg的格式解码从而得到图像对应的三维矩阵。
    # 解码之后的结果为一个张量，在使用它的取值之前需要明确调用运行的过程。
    img_data = tf.image.decode_jpeg(image_raw_data)
    
    # 输出解码之后的三维矩阵及其shape（即图片大小和色彩方式）
    print('shape: ', img_data.eval().shape, '\n', img_data.eval())
    
    # 使用pyplot工具可视化得到的图像
    plt.imshow(img_data.eval())
    plt.show()
    
    # 将表示一张图像的三维矩阵重新按照jpeg格式编码并存入文件中。
    encoded_image = tf.image.encode_jpeg(img_data)
    with tf.gfile.GFile(name='cat_encoded.jpg', mode="wb") as f:
        f.write(encoded_image.eval())

#### 图像大小调整


图像大小调整有两种方式，**第一种是通过算法使得新的图像尽量保存原始图像上的所有信息**。TensorFlow提供了4种不同的方法，并且将它们封装到了tf.image.resize_ images函数。

In [ ]:
with tf.Session() as sess:
    # 如果直接以0-255范围的整数数据输入resize_images，那么输出将是0-255之间的实数，
    # 不利于后续处理。建议在调整图片大小前，先将图片转为0-1范围的实数。
    image_float = tf.image.convert_image_dtype(img_data, tf.float32)
    print(image_float.eval())
    
    # 通过tf.image.resize_images函数调整图像的大小。这个函数第一个参数为原始图像，
    # 第二个参数为调整后图像的大小，method参数给出了调整图像大小的算法。
    # 注意，如果输入数据是unit8格式，那么输出将是0～255之内的实数，不方便后续处理。
    # 建议在调整图像大小前先转化为实数类型。
    methods = ['双线性插值法', '最近邻插值法', '双三次插值法', '面积插值法']
    for i in range(4):
        print('interpolation method: ', methods[i])
        resized = tf.image.resize_images(image_float, [300, 300], method=i)
        resized = tf.clip_by_value(resized, 0.0, 1.0)   # clip到0-1之间
        plt.imshow(resized.eval())
        plt.show()

tf.image.resize_images函数中method参数取值与相对应的图像大小调整算法：  
0：双线性插值法  
1：最近邻居法  
2：双三次插值法  
3：面积插值法

不同于第一种图像调整方法的会保存完整图像信息，**第二种调整方法是对图像进行剪裁或填充**。  
tf.image.resize_image_with_crop_or_pad函数调整图像大小：

In [ ]:
with tf.Session() as sess:    
    # 自动截取原始图像居中部分
    croped = tf.image.resize_image_with_crop_or_pad(img_data, 1000, 1000)
    # 自动在原始图像四周填充全0背景
    padded = tf.image.resize_image_with_crop_or_pad(img_data, 3000, 3000)
    plt.imshow(croped.eval())
    plt.show()
    plt.imshow(padded.eval())
    plt.show()

tf.image.central_crop通过比例调整图像大小：

In [ ]:
with tf.Session() as sess:   
    # 第一个参数为原始图像，第二个为调整比例
    central_cropped = tf.image.central_crop(img_data, 0.5)
    plt.imshow(central_cropped.eval())
    plt.show()

tf.image.crop_to_bounding_box函数和tf.image.pad_to_bounding_ box函数可以剪裁或者填充给定区域的图像。具体参见TensorFlow的API文档。

#### 图像翻转
在很多图像识别问题中，图像的翻转不应该影响识别的结果。随机地翻转训练图像，训练得到的模型可以识别不同角度的实体，因此**随机翻转是常用的图像预处理方式**。

In [ ]:
with tf.Session() as sess: 
    # 上下翻转
    flipped = tf.image.flip_up_down(img_data)
    # 左右翻转
    flipped = tf.image.flip_left_right(img_data)
    #对角线翻转
    flipped = tf.image.transpose_image(img_data)
    # 以一定概率上下翻转图片。
    flipped = tf.image.random_flip_up_down(img_data)
    # 以一定概率左右翻转图片。
    flipped = tf.image.random_flip_left_right(img_data)
    
    plt.imshow(flipped.eval())
    plt.show()

#### 图像色彩调整
调整图像的亮度(brightness)、对比度(contrast)、色相(hue)和饱和度(saturation)在很多图像识别应用中都不应该影响识别结果。可以随机调整训练图像的这些属性，从而使训练得到的模型尽可能小地受到无关因素的影响。  
色彩调整的API可能导致像素的实数值超出0.0-1.0的范围，因此在输出最终图像前需要将其值截断在0.0-1.0范围区间，否则不仅图像无法正常可视化，以此为输入的神经网络的训练质量也可能受到影响。如果对图像进行多项处理操作，那么这一截断过程应当在所有处理完成后进行。

In [ ]:
# 修改图像的亮度(brightness)、对比度(contrast)
with tf.Session() as sess:
    # 在进行一系列图片调整前，先将图片转换为实数形式，有利于保持计算精度。
    image_float = tf.image.convert_image_dtype(img_data, tf.float32)
    # 将图片的亮度-0.5。
    adjusted = tf.image.adjust_brightness(image_float, -0.5)
    # 将图片的亮度+0.5
    adjusted = tf.image.adjust_brightness(image_float, 0.5)
    # 在[-max_delta, max_delta)的范围随机调整图片的亮度。
    adjusted = tf.image.random_brightness(image_float, max_delta=0.5)
    
    # 将图片的对比度减少到0.5倍
    adjusted = tf.image.adjust_contrast(image_float, 0.5)
    # 将图片的对比度+5倍
    adjusted = tf.image.adjust_contrast(image_float, 5)
    # 在[lower, upper]的范围随机调整图的对比度。
    adjusted = tf.image.random_contrast(image_float, lower, upper)

    # 在最终输出前，将实数取值截取到0-1范围内。
    adjusted = tf.clip_by_value(adjusted, 0.0, 1.0)
    plt.imshow(adjusted.eval())
    plt.show()

In [ ]:
# 修改图像的色相(hue)和饱和度(saturation)
with tf.Session() as sess:
    # 在进行一系列图片调整前，先将图片转换为实数形式，有利于保持计算精度。
    image_float = tf.image.convert_image_dtype(img_data, tf.float32)
    # 分别将色相加0.1、0.3、0.6、0.9
    adjusted = tf.image.adjust_hue(image_float, 0.1)
    adjusted = tf.image.adjust_hue(image_float, 0.3)
    adjusted = tf.image.adjust_hue(image_float, 0.6)
    adjusted = tf.image.adjust_hue(image_float, 0.9)
    # 在[-max_delta, max_delta]的范围随机调整图片的色相。max_delta的取值在[0, 0.5]之间。
    adjusted = tf.image.random_hue(image_float, max_delta)
    
    # 将图片的饱和度-5。
    adjusted = tf.image.adjust_saturation(image_float, -5)
    # 将图片的饱和度+5。
    adjusted = tf.image.adjust_saturation(image_float, 5)
    # 在[lower, upper]的范围随机调整图的饱和度。
    adjusted = tf.image.random_saturation(image_float, lower, upper)
    
    # 在最终输出前，将实数取值截取到0-1范围内。
    adjusted = tf.clip_by_value(adjusted, 0.0, 1.0)
    plt.imshow(adjusted.eval())
    plt.show()

TensorFlow还提供API来完成图像标准化的过程——将图像上的亮度均值变为0，方差变为1：

In [ ]:
with tf.Session() as sess:
    # 在进行一系列图片调整前，先将图片转换为实数形式，有利于保持计算精度。
    image_float = tf.image.convert_image_dtype(img_data, tf.float32)
    
    # 将代表一张图片的三维矩阵中的数字均值变为0，方差变为1。
    adjusted = tf.image.per_image_standardization(image_float)
    
    # 在最终输出前，将实数取值截取到0-1范围内。
    adjusted = tf.clip_by_value(adjusted, 0.0, 1.0)
    plt.imshow(adjusted.eval())
    plt.show()

#### 处理标注框
tf.image.draw_bounding_boxes函数处理标注框:

In [ ]:
with tf.Session() as sess:     
    # 在原图上用标注框画出截取的范围。由于原图的分辨率较大（2673x1797)，生成的标注框 
    # 在Jupyter Notebook上通常因边框过细而无法分辨，这里为了演示方便先缩小分辨率。
    image_small = tf.image.resize_images(img_data, [180, 267], method=1)
    
    # tf.image.draw_bounding_boxes要求输入图片必须是实数类型。
    image_float = tf.image.convert_image_dtype(image_small, tf.float32)
    # 第一个参数是一个batch的数据，也就是四维的；
    # 第二个参数是实数型，0-1之间，是相对值
    batchced_img = tf.expand_dims(image_float, 0)
    # 代表了从（63，125）到（90，150）的图像
    boxes = tf.constant([[[0.05, 0.05, 0.9, 0.7], [0.35, 0.47, 0.5, 0.56]]])
    result = tf.image.draw_bounding_boxes(batchced_img, boxes)
    plt.imshow(result[0].eval())
    plt.show()

**随机截取图像上有信息含量的部分**也是一个提高模型健壮性（robustness）的一种方式，可以使训练得到的模型不受被识别物体大小的影响。  
tf.image.sample_distorted_bounding_ box函数完成随机截取图像的过程:

In [ ]:
with tf.Session() as sess:         
    boxes = tf.constant([[[0.05, 0.05, 0.9, 0.7], [0.35, 0.47, 0.5, 0.56]]])
    
    # sample_distorted_bounding_box要求输入图片必须也是实数类型。
    image_float = tf.image.convert_image_dtype(img_data, tf.float32)
    # min_object_covered=0.4表示截取部分至少包含某个标注框40％的内容
    begin, size, bbox_for_draw = tf.image.sample_distorted_bounding_box(
        tf.shape(image_float), bounding_boxes=boxes, min_object_covered=0.4)
    
    # 截取后的图片
    distorted_image = tf.slice(image_float, begin, size)
    plt.imshow(distorted_image.eval())
    plt.show()

    # 在原图上用标注框画出截取的范围
    image_small = tf.image.resize_images(image_float, [180, 267], method=0)
    batchced_img = tf.expand_dims(image_small, 0)
    image_with_box = tf.image.draw_bounding_boxes(batchced_img, bbox_for_draw)
    print(bbox_for_draw.eval())
    plt.imshow(image_with_box[0].eval())
    plt.show()

TensorFlow提供的主要图像处理函数小结：  
●图像编码处理。tf.image.decode_jpeg、tf.image.decode_png、tf.image.encode_jpeg、tf.image.encode_png 。  
●图像大小调整。保存全部信息的tf.image.resize_ images、对图像剪裁或填充的tf.image.resize_image_with_crop_or_pad、比例调整tf.image.central_crop。  
●图像反转。tf.image.flip_up_down、tf.image.flip_left_right(img_data)、tf.image.transpose_image以及前两者的random。  
●图像色彩调整。tf.image.adjust_brightness、tf.image.adjust_contrast、tf.image.adjust_hue、tf.image.adjust_saturation以及四者的random，以及图像标准化tf.image.per_image_whitening。  
●处理边框&随机截取。tf.image.draw_bounding_boxes、tf.image.sample_distorted_bounding_box。  
另外，读取源文件是tf.gfile.FastGFile(name="../../datasets/cat.jpg", mode="rb").read()，保存文件是tf.gfile.GFile(name='cat_encoded.jpg', mode="wb").write()。  
在解决真实的图像识别问题时，一般会使用多种处理方法。

下面程序完成了**从图像片段截取，到图像大小调整，再到图像反转，最后图像色彩调整**的整个图像预处理过程。这样就可以从一张图片衍生出很多训练样本，通过将训练图片进行预处理，得到神经网络模型可以识别不同大小、方位、色彩等方便的实体。

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


# 给定一张图像，随机调整图像的色彩。因为调整亮度、对比度、饱和度和色相的顺序会影
# 最后得到的结果，所以可以定义多种不同的顺序。具体使用哪一种顺序可以在训练数据
# 预处理时随机地选择一种。这样可以进一步降低无关因素对模型的影响。
def distort_color(image, color_ordering=0):
    if color_ordering == 0:
        image = tf.image.random_brightness(image, max_delta=32./255.)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.image.random_hue(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    else:
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.image.random_brightness(image, max_delta=32./255.)
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
        image = tf.image.random_hue(image, max_delta=0.2)
    # 还可以定义其它排列，不一一列出

    return tf.clip_by_value(image, 0.0, 1.0)


# 给定一张解码后的图像、目标图像的尺寸以及图像上的标注框，此函数可以对给出的图像进行顶
# 处理。这个函数的输入图像是图像识别问题中原始的训练图像，而输出则是神经网络模型的输入
# 层。注意这里只处理模型的训练数据，对于预测的数据，一般不需要使用随机变换的步骤。
def preprocess_for_train(image, height, width, bbox):
    # 查看是否存在标注框
    if bbox is None:
        bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])
    # 转换图像张量的类型
    if image.dtype != tf.float32:
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        
    # 随机的截取图片中一个块
    bbox_begin, bbox_size, _ = tf.image.sample_distorted_bounding_box(
        tf.shape(image), bounding_boxes=bbox, min_object_covered=0.4)
    distorted_image = tf.slice(image, bbox_begin, bbox_size)

    # 将随机截取的图片调整为神经网络输入层的大小。大小调整的算法时随机的
    distorted_image = tf.image.resize_images(distorted_image, [height, width], 
                                             method=np.random.randint(4))
    # 随机左右翻转
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    # 使用一种随机的顺序调整图像色彩
    distorted_image = distort_color(distorted_image, np.random.randint(2))
    return distorted_image


image_raw_data = tf.gfile.FastGFile("../../datasets/cat.jpg", "rb").read()
with tf.Session() as sess:
    img_data = tf.image.decode_jpeg(image_raw_data)
    boxes = tf.constant([[[0.05, 0.05, 0.9, 0.7], [0.35, 0.47, 0.5, 0.56]]])
    for i in range(9):
        result = preprocess_for_train(img_data, 299, 299, boxes)
        plt.imshow(result.eval())
        plt.show()